In [33]:
import util
import model
import tensorflow as tf
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, make_scorer
from tensorflow.keras import datasets, layers, models, callbacks, regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [34]:
# Instantiate sequential with the name model

# Specify number of epochs for the two models
epochs_drop = 10
epochs_l2 = 10

# Specify K for K-fold cross validaton
num_folds = 5

In [35]:
train_images, train_labels, test_images, test_labels, image_shape = util.get_images()

model_drop = model.model_drop_init(image_shape)
model_l2 = model.model_l2_init(image_shape)

# Output summaries of each of the models
model_drop.summary()
model_l2.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_16 (Flatten)         (None, 1024)              0         
_________________________________________________________________
dense_32 (Dense)             (None, 64)              

In [36]:
# Compile both models

model_drop.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model_l2.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [37]:
stopper = callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

In [40]:
print("Dropoff\n")
classifier_drop = KerasClassifier(
    model.model_drop_init,
    image_shape=image_shape,
    epochs=epochs_drop,
    batch_size=32,
)

dropScores = cross_validate(
    estimator=classifier_drop, X=train_images, y=train_labels, cv=num_folds, scoring=make_scorer(accuracy_score)
)
print(dropScores["test_score"])

Dropoff



/common/cse478/shared/envs/hw/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/common/cse478/shared/envs/hw/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/common/cse478/shared/envs/hw/lib/python3.8/site-packages/tensorflow/python/keras/wrappers/scikit_learn.py", line 223, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/common/cse478/shared/envs/hw/lib/python3.8/site-packages/tensorflow/python/keras/wrappers/scikit_learn.py", line 159, in fit
    if (losses.is_categorical_crossentropy(self.model.loss) and
AttributeError: 'Sequential' object has no attribute 'loss'

  warnings.warn("Estimator fit failed. The score on this train-test"
/common/cse478/shared/envs

NotFittedError: All estimators failed to fit

In [ ]:
# Plot accuracies vs epoch for model_drop
plt.plot(history_drop.history["accuracy"], label="accuracy")
plt.plot(history_drop.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim([0.5, 1])
plt.title("model_drop accuracy vs epochs")
plt.legend(loc="lower right")
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

In [ ]:
print("L2\n")
l2Scores = cross_val_score(
    estimator=model_l2, X=train_images, y=train_labels, cv=kfold
)
print(l2Scores["test_score"])

In [ ]:
# Plot accuracies vs epoch for model_l2
plt.plot(history_l2.history["accuracy"], label="accuracy")
plt.plot(history_l2.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim([0.5, 1])
plt.title("model_l2 accuracy vs epochs")
plt.legend(loc="lower right")
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)